In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import Adam

In [ ]:
train = pd.read_csv('../input/Kannada-MNIST/train.csv')
train.head()

In [ ]:
y_train = np.array(train['label'])
y_train

In [ ]:
X_train = train.drop(['label'],1)
X_train.head()

In [ ]:
X_train = np.array(X_train).reshape(X_train.shape[0],28,28)
X_train.shape

In [ ]:
plt.imshow(X_train[0])

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, 10)
print(y_train[:10])

In [ ]:
X_train = np.array(X_train/255)

from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid=train_test_split(X_train, y_train, test_size=0.2)

x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_valid=x_valid.reshape(x_valid.shape[0],28,28,1)



# define the model

model = Sequential()

model.add(Conv2D(input_shape=(28, 28, 1),filters=16, kernel_size=3, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.1))
model.add(Dropout(0.1))

model.add(Conv2D(32, kernel_size=3, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.15))
model.add(Dropout(0.15))
model.add(MaxPool2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=5, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.17))
model.add(MaxPool2D(pool_size=2))
#model.add(Dropout(0.3))

model.add(Conv2D(filters=128, kernel_size=5, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.15))
model.add(Dropout(0.17))

model.add(Conv2D(filters=64, kernel_size=3, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.15))
#model.add(MaxPool2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=3, padding='SAME', activation='relu'))
model.add(BatchNormalization(momentum=0.15))
model.add(Dropout(0.05))

model.add(Flatten())
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.05))
model.add(Dense(25, activation = "relu"))
model.add(Dropout(0.03))
model.add(Dense(10, activation = "softmax"))

model.summary()

In [ ]:
from keras.preprocessing. image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint('bestweights.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode="max")

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

batch_size=128
epochs = 60




datagenerator=ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (deg 0 to 180)
        zoom_range=0.25,
        shear_range=0.25,
        width_shift_range=0.2,  # randomly shift images horizontally
        height_shift_range=0.2,  # randomly shift images vertically
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagenerator.fit(x_train)


bm=model.fit_generator(datagenerator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,validation_data=(x_valid, y_valid), verbose=2,steps_per_epoch=x_train.shape[0]//batch_size, callbacks=[checkpoint])
loss, accuracy = model.evaluate_generator(datagenerator.flow(x_valid, y_valid, batch_size=batch_size), verbose=2, callbacks=[checkpoint])
model.load_weights("bestweights.hdf5")

In [ ]:
test = pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
test = test.drop(['id'],1)

In [ ]:
X_test = np.array(test/255)

X_test = X_test.reshape(X_test.shape[0],28,28)

X_test=X_test.reshape(X_test.shape[0],28,28,1)

X_test.shape

In [ ]:
results=model.predict(X_test)

In [ ]:
results=np.argmax(results, axis=1)

In [ ]:
results

In [ ]:
cnn3 = pd.DataFrame({'label':results}).reset_index().rename(columns = {'index':'id'})
cnn3.head()

In [ ]:
cnn3.to_csv('cnn3_img_aug.csv',index = False)

In [ ]:
# list all data in history
print(bm.history.keys())
# summarize history for accuracy
plt.plot(bm.history['accuracy'])
plt.plot(bm.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(bm.history['loss'])
plt.plot(bm.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()